<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L5_tasks_event_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L5: Automate Event Planning

This Jupyter notebook demonstrates an event planning workflow using AI agents. The workflow involves various tasks such as identifying and booking a venue, managing logistics, and promoting the event. It uses the crewai library to create and manage AI agents that coordinate these tasks.

In this lesson, you will learn more about Tasks.

## Library Installation

In [1]:
!pip install crewai==0.28.8
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 806.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!pip install crewai_tools==0.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 899.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Warnings Suppression

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Importing Required Classes

In [4]:
import os
from google.colab import userdata
from crewai import Agent, Crew, Task

## Setting API Keys and LLM

In [6]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Defining Tools

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Defining Agents

###  Venue Coordinator

In [9]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

### Logistics Manager

In [10]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

### Marketing and Communication Agent

In [11]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [12]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Defining Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

### Venue Task

In [13]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

### Logistics Task

In [14]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,     # the task can run in parallel with the tasks which come after it
    agent=logistics_manager
)

### Marketing Task

In [15]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Assembling the Crew and Executing the Workflow

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [16]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for the same input, so what you get might be different than what you see in the video.

**Note 2**:
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [18]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: To begin the process of finding a suitable venue for the Tech Innovation Conference in San Francisco, I need to search the internet to gather comprehensive information on available venues that can accommodate such an event.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in San Francisco 2023"} 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: The search attempt resulted in an unauthorized access error. I will try a different approach, perhaps by focusing on a broader category of venues suitable for technology conferences in San Francisco, rather than a specific event.

Action: Search the internet
Action Input: {"search_query": "Technology conference venues in San Francisco"} 

{'message': 'Unauthorized.', 'statusCode': 403}



## Displaying Results
- Display the generated `venue_details.json` file.

### Venue Details

In [19]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco',
 'booking_status': 'available',
 'capacity': 0,
 'name': 'The Moscone Center'}


> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

### Marketing Report

In [21]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Plan for the Tech Innovation Conference

## Venue Details
- **Location:** The Moscone Center, San Francisco
- **Capacity:** Large-scale venue ideal for hosting significant tech events

## Promotion Strategies
### 1. **Diverse Ticketing Options**
   - Offer a range of ticket types to cater to different attendees, such as early-bird tickets, standard tickets, and VIP passes.
   - Provide discounts for students, startups, or groups to encourage broader participation.

### 2. **Leverage Sponsorship Opportunities**
   - Collaborate with tech companies and startups for event sponsorship, providing them visibility in exchange for financial support or services.
   - Create tiered sponsorship packages with varying levels of benefits, such as branding, exhibition space, and speaking opportunities.

### 3. **Strategic Event Marketing**
   - Utilize social media platforms (Facebook, LinkedIn, Twitter) to reach a wider audience. Regular updates, behind-the-scenes content, and speaker announcements can generate buzz.
   - Implement an email marketing campaign to send out newsletters with key information, updates, and reminders to potential attendees.
   - Engage with tech blogs and influencers to promote the event through articles, interviews, and partnered content.

### 4. **Hosting Pre-Event Activities**
   - Organize webinars or live Q&A sessions with key speakers to pique interest and provide value ahead of the event.
   - Arrange meetups or small gatherings in different cities to build community and spread word-of-mouth awareness.

## Engagement Strategies
### 1. **Interactive Platforms**
   - Develop an event app that provides schedules, speaker bios, networking opportunities, and real-time notifications.
   - Utilize polling and Q&A features during sessions to increase attendee interaction and engagement.

### 2. **Networking Opportunities**
   - Designate networking zones within the venue where attendees can connect with peers, mentors, and industry leaders.
   - Schedule structured networking sessions or speed networking to facilitate new connections.

### 3. **Follow-up Post Event**
   - Send thank-you emails to attendees with highlights and key takeaways from the conference.
   - Share recordings or summaries of sessions for those who could not attend or wish to revisit the content.
   - Solicit feedback through surveys to improve future events and maintain engagement with the community.

By implementing these strategies, the Tech Innovation Conference is set to not only attract at least 500 potential attendees but also provide them with a valuable and memorable experience.

# Conclusion
This notebook demonstrates how to automate event planning using a multi-agent system, ensuring the coordination of various tasks like venue booking, logistics management, and marketing.